<a href="https://colab.research.google.com/github/Suleymanabdy/Data-Science-Checkpoints./blob/main/Creating_Your_Own_Chatbot_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Installing the required packages
!pip install streamlit
!pip install pyngrok
!pip install streamlit-option-menu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 829.3/829.3 kB 8.7 MB/s eta 0:00:00


In [ ]:
%%writefile app.py
# Import necessary libraries
import streamlit as st                     # Streamlit to build the app's interface
import pandas as pd                        # Pandas for data handling
import numpy as np                         # Numpy for numerical computations
from sklearn.model_selection import train_test_split  # For splitting data into training and test sets
from sklearn.preprocessing import StandardScaler      # For scaling numerical features
from sklearn.ensemble import RandomForestClassifier   # Random Forest model for classification

# Function to load the brain tumor dataset
def load_data():
    # Update the correct path or URL for your Brain Tumor dataset
    data = pd.read_csv('/content/Brain Tumor.csv')

    # Check the shape and first few rows of the dataset
    st.write(f"Dataset Shape: {data.shape}")
    st.write("First few rows of the dataset:")
    st.write(data.head())

    # Assuming the dataset has 15 columns, let's print the column names
    st.write(f"Dataset columns: {data.columns.tolist()}")

    # Update column names based on the dataset's actual structure (you can inspect them above)
    column_names = ["Class", "Mean", "Variance", "Standard Deviation", "Entropy", "Skewness", "Kurtosis",
                    "Contrast", "Energy", "Feature10", "Feature11", "Feature12", "Feature13", "Feature14", "Feature15"]

    if data.shape[1] == len(column_names):
        data.columns = column_names
    else:
        st.warning("The number of columns in the dataset does not match the expected number. Please adjust the column names.")

    # Replace '?' with NaN
    data = data.replace('?', np.nan)

    # Convert all columns to numeric
    for column in data.columns:
        data[column] = pd.to_numeric(data[column], errors='coerce')

    # Fill missing values with median for numeric columns
    for column in data.columns:
        data[column] = data[column].fillna(data[column].median())

    return data

# Function to train the Random Forest model
def train_model(data):
    X = data.drop('Class', axis=1)       # Dropping the target column (dependent variable)
    y = data['Class'].apply(lambda x: 1 if x > 0 else 0)  # Converting the target to binary (1: tumor, 0: no tumor)

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Scaling the feature variables
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)

    # Training a Random Forest classifier
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train_scaled, y_train)

    return model, scaler  # Return the trained model and the scaler for later use

# Main function to define the Streamlit app layout
def main():
    # Set the app title and introductory text
    st.title("Brain Tumor Detection Assistant")
    st.write("Your interactive guide to detecting brain tumors from medical data")

    # Load and train the model
    @st.cache_resource
    def load_trained_model():
        data = load_data()
        model, scaler = train_model(data)
        return model, scaler, data

    # Load the model, scaler, and data
    model, scaler, data = load_trained_model()

    # Create two tabs: Chat Assistant and Tumor Detection
    tab1, tab2 = st.tabs(["Chat Assistant", "Tumor Detection"])

    with tab1:
        st.subheader("Ask me about brain tumors!")
        user_query = st.selectbox("Choose your question:", ["What is a brain tumor?", "What are the symptoms?", "How can I detect it?"])
        if st.button("Get Answer"):
            if user_query == "What is a brain tumor?":
                st.write("A brain tumor is an abnormal growth of cells within the brain that can be cancerous or non-cancerous.")
            elif user_query == "What are the symptoms?":
                st.write("Common symptoms include headaches, vision problems, and seizures.")
            else:
                st.write("Brain tumors can be detected through medical imaging, such as MRI or CT scans.")

    with tab2:
        st.subheader("Brain Tumor Detection")
        st.write("Enter the features of the brain tumor data for detection:")

        # Input fields for tumor data details
        mean = st.number_input("Mean", -10.0, 10.0, 0.0)
        variance = st.number_input("Variance", 0.0, 10.0, 0.0)
        std_dev = st.number_input("Standard Deviation", 0.0, 10.0, 0.0)
        entropy = st.number_input("Entropy", -10.0, 10.0, 0.0)
        skewness = st.number_input("Skewness", -10.0, 10.0, 0.0)
        kurtosis = st.number_input("Kurtosis", -10.0, 10.0, 0.0)
        contrast = st.number_input("Contrast", 0.0, 10.0, 0.0)
        energy = st.number_input("Energy", 0.0, 10.0, 0.0)

        # Assess risk on button click
        if st.button("Detect Tumor"):
            input_data = np.array([[mean, variance, std_dev, entropy, skewness, kurtosis, contrast, energy]])
            scaled_data = scaler.transform(input_data)  # Scale the input data
            prediction = model.predict(scaled_data)  # Get the model's prediction
            st.write("The model predicts: " + ("Tumor Detected" if prediction == 1 else "No Tumor Detected"))

if __name__ == "__main__":
    main()


Overwriting app.py


In [ ]:
from pyngrok import ngrok

# Install Ngrok authtoken
!ngrok authtoken 2rtI1iz9mTRdDm5G9nixP8XT3lS_25ywsP6gShfGUuK3RhfMW

# Run Streamlit app
!streamlit run app.py &>/dev/null&

# Create an Ngrok tunnel for port 8501 (Streamlit default port)
public_url = ngrok.connect(8501, "http")

# Print the public URL to access the app
print(public_url)


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
NgrokTunnel: "https://a856-34-136-63-63.ngrok-free.app" -> "http://localhost:8501"
